# Test Notebook illustrating Bug in bqplot and voici

When this Jupyter notebook is run in Voila, via the command line

```bash
voila webapp/index.ipynb
```
 it works correctly. Specifically dragging the points on the plot not only updates the location of the point but the line connecting the points.
 
 However, if you create a static web app with Voici with the command:
 ```bash
voici build --contents webapp
```
and serve the static output directory with the command:
```bash
python -m http.server 8000 --directory _output
```
The resulting `index.ipynb` page at [`http://localhost:8000/`](http://localhost:8000/) does NOT function properly.  You should be able to see that the background line connecting the points does not update when the scatter points are dragged.

In [ ]:
# Notebook to test Scatter and moving points in Voici

from IPython.display import display
import pandas as pd
import numpy as np
import bqplot as bq
import ipywidgets as widgets

In [ ]:
##
## Define functions for interactive plot with moveable points
##

# Function to react to changes in the scatter plot
def update_TP(change):
    global TPline, TPscat

    # update line on screen
    TPline.y = TPscat.y

##
## Adjust scales/axes/tooltips as necessary from the previous plot
##

# Scales to transform the data
min_lum = 10**-4
max_lum = 10**4

x_scale = bq.LinearScale(min =-1, max=2)
y_scale = bq.LogScale(min = min_lum, max=max_lum)

# Labels and scales for Axes
ax_x = bq.Axis(label='X value', scale=x_scale, tick_format='.1f', num_ticks=7)
ax_y = bq.Axis(label='Y value', scale=y_scale, orientation='vertical')

# moving the label perpendicular to the axis
ax_y.label_offset = '3em'

# Create an editable line with scatter plot
TPx = np.linspace(-0.5, 1.5, 11)
TPy = np.ones_like(TPx)    # Initial flat TP curve
opaque = np.ones_like(TPx) # Initial opacity of points

# Create a scatter plot with editable points
TPscat = bq.Scatter(x=TPx, y=TPy, color=None,
                    marker='circle',
                    scales={'x': x_scale, 'y': y_scale},
                    default_size=50,
                    colors=['darkturquoise'],
                    default_opacities=opaque,
                    enable_move=True, restrict_y = True, continuous_update=True)

# Generate a line plot from the scatter plot
TPline = bq.Lines(x=TPscat.x, y=TPscat.y,
                  scales={'x': x_scale, 'y': y_scale},
                  default_opacities=opaque,
                  colors=['darkturquoise'])

In [ ]:
## Create a test plot meant to allow user to move scatter plot
## points around and have the line adjust to match the points

# Create a Figure from the scatter plot and line
test_figure = bq.Figure(axes=[ax_x, ax_y],
                        marks=[TPline, TPscat],
                        animation_duration = 250,
                        title='Test Figure (Try Dragging Points)',
                        background_style={'fill': 'black'},
                        layout={'width': '400px', 'min_height': '400px'})

# Observe the scatter plot points to see if I moved one
TPscat.observe(update_TP, names=['y'])

In [ ]:
display(test_figure)